# Eritrea Twitter Lab 4

name: Coleman Shepard <br>
student_id: shepa135

### Twitter Authentication Information

In [1]:
# Insert your consumer key and secret inside the quotes below
consumer_key = "p4jlC8LHlrVHA1JTwz95ngG49"
consumer_secret = "zDwdAl2Rks3gqwHwQ2SMyb8IlWPYvJfbHP0bqJ89PBsUuN0NsM"

# Insert your access key and secret inside the quotes below
access_token_key = "834119282153943044-jHuQpMCD8LzuYBevUFkjyPBmDaMIExz"
access_token_secret = "NVjJxNOQ70mz0xGi8J48evLxVIXuXtx1v2MMtIE0frD4q"

print("Consumer and access keys and secrets have been successfully added")


Consumer and access keys and secrets have been successfully added


### Import Modules

In [2]:
# Example: Search for and print tweets using the 'search/tweets' endpoint 

# This line allows us to access the TwitterAPI library
from TwitterAPI import TwitterAPI
import pprint
import psycopg2
import time

### Connect and Create a Cursor for a Postgres Database

Instead of writing the data collected from Twitter to a csv file, I chose to write the streamed data to a postgres database using the psycopg2 module in Python. This way I do not have to worry about any cocurrency issues when I query data from the database if data is still being writing to it. This also allows multiple users/scripts to write to the database which could be useful with multiple people working on the same project. The database is locally hosted (At the moment), so passwords and other information are close to default. If the database was hosted somewhere, I would explore the security measures that need to be in place. 

In [ ]:
# Creates a connection to a database in Postgres (Local)
twitter_connection = psycopg2.connect(user = "tweet",
                                  password = "ggez",
                                  host = "127.0.0.1",
                                  port = "5433",
                                  database = "twitter")

# Needed for connection
cursor = twitter_connection.cursor()

# Print PostgreSQL Connection properties
print (twitter_connection.get_dsn_parameters(),"\n")

## Create a table in the database

A database called twitter was created through PgAdmin, but I wanted to practice creating tables with desired schemas in Python. At this point, I just assigned a Primary Key which will correspond to a counter variable running the stream for tweets. Right now, I'm not sure if this is the best way to go about this, especially if I were to index it. A clustered index around hashtags may prove to be better for my goal. At the moment, I'm just worried about getting going with Postgres & Python. I may make alterations down the line to optimize the retrieval of information. 

### Schema Creation

In [ ]:
# Schema for Postgres Table Information Collection
Table = """CREATE TABLE twitter_t
                (ID INT PRIMARY KEY NOT NULL,
                TIME REAL,
                NAME TEXT[],
                FULL_TEXT TEXT[],
                HASHTAG TEXT[],
                COORDINATE TEXT,
                GEO TEXT,
                PLACE TEXT,
                FAV_COUNT INT,
                QUOTE_COUNT INT,
                REPLY_COUNT INT,
                RETWEET_COUNT INT,
                STATUS_COUNT INT,
                SCREEN_NAME TEXT,
                FRIENDS_COUNT INT);"""

### Execute Table Command 

In [ ]:
# Execute CREATE TABLE command
cursor.execute(Table)

# Commit the action to the database
twitter_connection.commit()

# Close the cursor
twitter_connection.close()
print("Table created successfully in PostgreSQL ")

## Twitter Script

This is the script I use to gather Twitter data and write it out to a record in my database. 

In [ ]:

# Creates a connection to a database in Postgres (Local)
twitter_connection = psycopg2.connect(user = "tweet",
                                  password = "ggez",
                                  host = "127.0.0.1",
                                  port = "5433",
                                  database = "twitter")
# Needed for connection
cursor = twitter_connection.cursor()


print("Starting TwitterAPI")
api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret,)

# Look for keywords in tweets
keywords = "eritrea","TPLF", "AbiyAhmed", "IsaiasAfewerki", "PFDJ","EPRDF","FreeEritrea", "IStandWithEritrea","#IStandWithEritrea","#Eritrea","HandsOffEritrea", "#HandsOffEritrea",

while True:
    try:
        print("Setting up filter request")
        # Notice we can add keyword filters (called 'track' term in Twitter)
        r = api.request('statuses/filter', {'track': keywords})

        # Count for database entry (Mistake)
        count = 4666


        print("Begin capturing tweets")
        for tweet in r:

            hashtags = [i['text'] + " "  for i in tweet['entities']['hashtags']]

            coords = tweet['coordinates']
        #     print(hashtags)
        #     print("ID",tweet['id'])
        #     print("TIME", tweet['created_at'])
        #     print("NAME",tweet['user']['name'])
        #     print("Full Text",tweet['text'])
        #     print("Hashtag",tweet['entities']['hashtags'])
            print("Coordinates",tweet['coordinates'])
            print("Geo",tweet['geo'])
            print("Place",tweet['place'])
        #     print("Fav_Count",tweet['favorite_count'])
        #     print("Quote_Count",tweet['quote_count'])
        #     print("Reply_Count",tweet['reply_count'])
        #     print("Retweet", tweet['retweet_count'])
        #     print("Statuses_Count", tweet['user']['statuses_count'])
            print("Screen_Name", tweet['user']['screen_name'])
            print("Friends_Count", tweet['user']['friends_count'])

            if tweet["retweeted"] == True:
                print("TRUE")

            if 'extended_tweet' in tweet:

                text = tweet['extended_tweet']['full_text']
            else:
                text = tweet['text']

            if 'coordinates' in tweet and tweet['coordinates'] != None:
                coords = str((tweet['coordinates']["coordinates"][0],tweet['coordinates']["coordinates"][1])) # Add the tweet coordinates to the list
                print(tweet['coordinates'])
            else:
                coords = tweet['coordinates']# If the tweet does not have coordinates then skip it


            if tweet['place'] != None:
                place = str(tweet['place']["full_name"]) # Add the tweet coordinates to the list

            else:
                place = tweet['place'] 





            cursor.execute(""" INSERT INTO twitter_t (ID, TIME, NAME, FULL_TEXT, HASHTAG,COORDINATE,GEO,PLACE,
                                                             FAV_COUNT,QUOTE_COUNT,REPLY_COUNT,RETWEET_COUNT,STATUS_COUNT,
                                                             SCREEN_NAME,FRIENDS_COUNT) VALUES (%s, %s, %s, %s, %s,
                                                             %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""" , (count, time.time(), [tweet['user']['name']],[text],
                                                                                                           hashtags, coords, tweet['geo'], place, 
                                                                                                           tweet['favorite_count'],tweet['quote_count'], tweet['reply_count'],
                                                                                                           tweet['retweet_count'],tweet['user']['statuses_count'], 
                                                                                                           [tweet['user']['screen_name']],tweet['user']['friends_count']))
            twitter_connection.commit()

            # Add 1 to count and if count is over 10 then stop capturing tweets
            count = count + 1
            print(count)
    
    except:
        pass
    
    time.sleep(15)

print("Done")

Starting TwitterAPI
Setting up filter request
Begin capturing tweets
Coordinates None
Geo None
Place None
Screen_Name danlntifada
Friends_Count 99
4667
Coordinates None
Geo None
Place None
Screen_Name arayawedihagher
Friends_Count 274
4668
Coordinates None
Geo None
Place None
Screen_Name TravelToEritrea
Friends_Count 547
4669
Coordinates None
Geo None
Place None
Screen_Name maria_wouden
Friends_Count 329
4670
Coordinates None
Geo None
Place None
Screen_Name Lucasseo11
Friends_Count 138
4671
Coordinates None
Geo None
Place None
Screen_Name TravelToEritrea
Friends_Count 547
4672
Coordinates None
Geo None
Place None
Screen_Name omegalll
Friends_Count 858
4673
Coordinates None
Geo None
Place None
Screen_Name HijiwunMekete
Friends_Count 529
4674
Coordinates None
Geo None
Place None
Screen_Name a_heerze
Friends_Count 101
4675
Coordinates None
Geo None
Place None
Screen_Name EritreaFirst
Friends_Count 508
4676
Coordinates None
Geo None
Place None
Screen_Name EritreaFirst
Friends_Count 508
467

Coordinates None
Geo None
Place None
Screen_Name EriEmnet
Friends_Count 65
4764
Coordinates None
Geo None
Place None
Screen_Name LorayMuhammad
Friends_Count 23
4765
Coordinates None
Geo None
Place None
Screen_Name EriEmnet
Friends_Count 65
4766
Coordinates None
Geo None
Place None
Screen_Name EriEmnet
Friends_Count 65
4767
Coordinates None
Geo None
Place None
Screen_Name ilusoaluso
Friends_Count 1000
4768
Coordinates None
Geo None
Place None
Screen_Name SamJenner99
Friends_Count 33
4769
Coordinates None
Geo None
Place None
Screen_Name ballamii
Friends_Count 623
4770
Coordinates None
Geo None
Place None
Screen_Name Eriitrea91
Friends_Count 337
4771
Coordinates None
Geo None
Place None
Screen_Name kingYacob1
Friends_Count 1344
4772
Coordinates None
Geo None
Place None
Screen_Name S_Sarkodee_Addo
Friends_Count 15
4773
Coordinates None
Geo None
Place None
Screen_Name kingYacob1
Friends_Count 1344
4774
Coordinates None
Geo None
Place None
Screen_Name MikeBates04
Friends_Count 719
4775
Coord

Coordinates None
Geo None
Place None
Screen_Name Latens_Verax
Friends_Count 868
4857
Coordinates None
Geo None
Place None
Screen_Name AdiamYo
Friends_Count 237
4858
Coordinates None
Geo None
Place None
Screen_Name semhabte
Friends_Count 256
4859
Coordinates None
Geo None
Place None
Screen_Name sammyonye3
Friends_Count 181
4860
Coordinates None
Geo None
Place None
Screen_Name BuschFrank
Friends_Count 2626
4861
Coordinates None
Geo None
Place None
Screen_Name good_whey
Friends_Count 28
4862
Coordinates None
Geo None
Place None
Screen_Name good_whey
Friends_Count 28
4863
Coordinates None
Geo None
Place None
Screen_Name UNHCRCanada
Friends_Count 1723
4864
Coordinates None
Geo None
Place None
Screen_Name CherryGuts
Friends_Count 136
4865
Coordinates None
Geo None
Place None
Screen_Name MagTeerey
Friends_Count 52
4866
Coordinates None
Geo None
Place None
Screen_Name jamesoreilly
Friends_Count 2081
4867
Coordinates None
Geo None
Place None
Screen_Name ForeverEritrea
Friends_Count 423
4868
Coo

Coordinates None
Geo None
Place None
Screen_Name patrickhooghuis
Friends_Count 66
4951
Coordinates None
Geo None
Place None
Screen_Name YonasManna
Friends_Count 1026
4952
Coordinates None
Geo None
Place None
Screen_Name appleoscar33
Friends_Count 5197
4953
Coordinates None
Geo None
Place None
Screen_Name yemanekeleta
Friends_Count 29
4954
Coordinates None
Geo None
Place None
Screen_Name AARMTL
Friends_Count 66
4955
Coordinates None
Geo None
Place None
Screen_Name colourbird00
Friends_Count 2528
4956
Coordinates None
Geo None
Place None
Screen_Name EllamellaMg
Friends_Count 447
4957
Coordinates None
Geo None
Place None
Screen_Name YonasManna
Friends_Count 1026
4958
Coordinates None
Geo None
Place None
Screen_Name OnlyTheSal
Friends_Count 245
4959
Coordinates None
Geo None
Place None
Screen_Name Volkskorrespdt
Friends_Count 251
4960
Coordinates None
Geo None
Place None
Screen_Name john_rorbach
Friends_Count 790
4961
Coordinates None
Geo None
Place None
Screen_Name 1966Paulap
Friends_Coun

Coordinates None
Geo None
Place None
Screen_Name 4MyPeers
Friends_Count 56
5040
Coordinates None
Geo None
Place None
Screen_Name mericolleoni
Friends_Count 242
5041
Coordinates None
Geo None
Place None
Screen_Name BLIKOPENER333
Friends_Count 18919
5042
Coordinates None
Geo None
Place None
Screen_Name Emnetian
Friends_Count 187
5043
Coordinates None
Geo None
Place None
Screen_Name GreatOkd
Friends_Count 601
5044
Coordinates None
Geo None
Place None
Screen_Name Bahlbiyemane
Friends_Count 327
5045
Coordinates None
Geo None
Place None
Screen_Name inamantjes
Friends_Count 422
5046
Coordinates None
Geo None
Place None
Screen_Name FloortjeBellefl
Friends_Count 2169
5047
Coordinates None
Geo None
Place None
Screen_Name Dun_Ad
Friends_Count 49
5048
Coordinates None
Geo None
Place None
Screen_Name Merkelhunter1
Friends_Count 738
5049
Coordinates None
Geo None
Place None
Screen_Name ErythreanSea
Friends_Count 830
5050
Coordinates None
Geo None
Place None
Screen_Name superpixel1961
Friends_Count 1

Coordinates None
Geo None
Place None
Screen_Name DennisChirp
Friends_Count 242
5138
Coordinates None
Geo None
Place None
Screen_Name romy_stern
Friends_Count 15
5139
Coordinates None
Geo None
Place None
Screen_Name BlutLeeuwarden
Friends_Count 217
5140
Coordinates None
Geo None
Place None
Screen_Name EremiasWmikael
Friends_Count 137
5141
Coordinates None
Geo None
Place None
Screen_Name MediaETHIOPIA
Friends_Count 304
5142
Coordinates None
Geo None
Place None
Screen_Name AiAfwerki
Friends_Count 84
5143
Coordinates None
Geo None
Place None
Screen_Name EritreaTweet
Friends_Count 1
5144
Coordinates None
Geo None
Place None
Screen_Name EritreaTweet
Friends_Count 1
5145
Coordinates None
Geo None
Place None
Screen_Name NationalEr_Int
Friends_Count 343
5146
Coordinates None
Geo None
Place None
Screen_Name Selkid
Friends_Count 221
5147
Coordinates None
Geo None
Place None
Screen_Name JimHaley17
Friends_Count 200
5148
Coordinates None
Geo None
Place None
Screen_Name BrhanAndai
Friends_Count 151
